In [2]:
import pyspark

In [5]:
from pyspark.sql import SparkSession


In [6]:
spark = SparkSession.builder.getOrCreate()



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/04 21:37:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [10]:
df.printSchema()


root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [11]:
df.repartition(4)

DataFrame[VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double]

In [12]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [13]:
df.write.parquet('yellow/2024/10')

In [21]:
filtered = df.filter(df.tpep_pickup_datetime == '2024-10-15')

In [22]:
df.count(filtered)

TypeError: DataFrame.count() takes 1 positional argument but 2 were given

In [23]:
filtered.show

<bound method DataFrame.show of DataFrame[VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double]>

In [24]:
filtered.count()

2

In [25]:
df.count()

3833771

In [26]:
filtered = df.filter(df.tpep_pickup_datetime == '2024-10-15')

In [27]:
filtered.count()

2

In [38]:
dates = ("2024-10-15 00:00:00",  "2024-10-15 23:59:59")


In [42]:
filtered = df.filter(df.tpep_pickup_datetime.between(*dates))

In [43]:
filtered.count()

128893

In [44]:
filtered.orderBy(filtered.tpep_pickup_datetime, ascending=False).show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-15 23:59:59|  2024-10-16 00:19:00|              1|         7.99|         1|                 N|         163|          13|           2|       33.8|  1.0|    0.5|       0.

In [45]:
from pyspark.sql.functions import unix_timestamp, col

In [51]:
df2 = df.withColumn("trip_duration_hours", 
                   (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600)

In [54]:
longest_trip = df2.orderBy(col("trip_duration_hours").desc()).limit(1)


In [55]:
longest_trip.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|trip_duration_hours|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------------+
|       2| 2024-10-16 13:03:49|  2024-10-23 07:40:53|              1|        32.37|         3|                 N|          48| 

In [56]:
from pyspark.sql.functions import col, count


In [57]:
pickup_counts = df.groupBy("PULocationID").agg(count("*").alias("pickup_count"))


In [58]:
least_frequent_pickup = pickup_counts.orderBy(col("pickup_count").asc()).limit(1)


In [59]:
least_frequent_pickup.show()

[Stage 32:========================================>               (10 + 4) / 14]

+------------+------------+
|PULocationID|pickup_count|
+------------+------------+
|         105|           1|
+------------+------------+



In [60]:
taxi_zones = spark.read.csv("taxi_zone_lookup.csv", header=True, inferSchema=True)

In [61]:
result = least_frequent_pickup.join(taxi_zones, least_frequent_pickup.PULocationID == taxi_zones.LocationID, "left") \
    .select("PULocationID", "pickup_count", "Borough", "Zone", "service_zone")

In [63]:
result.show()

+------------+------------+---------+--------------------+------------+
|PULocationID|pickup_count|  Borough|                Zone|service_zone|
+------------+------------+---------+--------------------+------------+
|         105|           1|Manhattan|Governor's Island...| Yellow Zone|
+------------+------------+---------+--------------------+------------+

